In [ ]:
#Importing libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import sklearn
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv("/content/train.csv")
X_test = pd.read_csv("/content/test.csv")

In [ ]:
y = data["label"]
X = data.drop(['label'],axis=1)
X

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(X.shape)
print(y.shape)

(42000, 784)
(42000,)


In [ ]:
#Converting dataframe to numpy array
X = np.array(X)
y = np.array(y)

In [ ]:
#Normalizing input
X = X/255

In [ ]:
#Train val split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, shuffle=True, random_state=1)

In [ ]:
#Making dummy variables for output
y_train = pd.get_dummies(y_train).values
y_val = pd.get_dummies(y_val).values
y_train

array([[0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
#Reshaping input for CNN input
X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_val = X_val.reshape(X_val.shape[0],28,28,1)

In [ ]:
#Creating CNN model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'same', activation = 'relu'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(256, activation= "relu"))
model.add(Dense(84, activation= "relu"))
model.add(Dense(32, activation= "relu"))
model.add(Dense(10, activation= "softmax"))

In [ ]:
adam = Adam(learning_rate = 5e-4)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

In [ ]:
#Data augmentation
datagen = ImageDataGenerator(
            rotation_range=10, 
            width_shift_range=0.1, 
            height_shift_range=0.1, 
            zoom_range=0.2)
datagen.fit(X_train)

In [ ]:
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        25632     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1568)              0         
                                                                 
 dense (Dense)               (None, 256)               4

In [ ]:
model.fit(datagen.flow(X_train, y_train, batch_size=500), steps_per_epoch=len(X_train)/500, epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
79/79 [==============================] - 75s 931ms/step - loss: 1.2025 - accuracy: 0.6169 - val_loss: 0.2682 - val_accuracy: 0.9352
Epoch 2/30
79/79 [==============================] - 74s 921ms/step - loss: 0.3569 - accuracy: 0.8932 - val_loss: 0.1816 - val_accuracy: 0.9467
Epoch 3/30
79/79 [==============================] - 75s 939ms/step - loss: 0.2395 - accuracy: 0.9284 - val_loss: 0.1230 - val_accuracy: 0.9676
Epoch 4/30
79/79 [==============================] - 75s 933ms/step - loss: 0.1828 - accuracy: 0.9457 - val_loss: 0.1252 - val_accuracy: 0.9657
Epoch 5/30
79/79 [==============================] - 75s 936ms/step - loss: 0.1554 - accuracy: 0.9534 - val_loss: 0.1005 - val_accuracy: 0.9757
Epoch 6/30
79/79 [==============================] - 74s 929ms/step - loss: 0.1404 - accuracy: 0.9580 - val_loss: 0.0962 - val_accuracy: 0.9733
Epoch 7/30
79/79 [==============================] - 74s 931ms/step - loss: 0.1170 - accuracy: 0.9647 - val_loss: 0.0840 - val_accuracy: 0.9790

In [ ]:
score = model.evaluate(X_val, y_val, batch_size=32)

NameError: ignored

In [ ]:
score

[0.0319855771958828, 0.989047646522522]